# Bert Model Fine-tuning

This notebook primarily utilizes the state-of-the-art (SOTA) Google BERT model, originally designed to learn how to order shuffled sentences within a paragraph. While most use cases focus on classification and Named Entity Recognition (NER), we often need to fine-tune this model to suit our specific needs.

One important consideration is that the context input length is limited to less than 512 tokens due to the model’s structural constraints. Additionally, high-quality training data is essential to achieve good generalization. We also test other spliting of dataset and we obtain the similar accuracy (Originally set to 42) but different trace of training loss. This highlights that the model learns something from the data.

Model used: BERT-base-multilingual-uncased

This [slides](https://docs.google.com/presentation/d/165GxBIZ-Jxk8uaEKK9h02GtFsKsCCQdm8ouar2JkCKU/edit?usp=sharing) has more details see here. 

In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler,
    random_split,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import (
    BertForSequenceClassification,
    AdamW,
    BertConfig,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)
import os

# Specify your cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/snt/miniconda3/envs/causal_env_transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

## Data For Training

In [23]:
dataset_path = "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/all_data_for_bert_training_augmented_opensource.csv"

# Load the dataset from a CSV file
dataset = pd.read_csv(dataset_path)
dataset["ability"] = dataset["ability"].replace({"SUM": "SUMM"})

# Convert the 'Rappel' column to a categorical type
dataset["ability"] = dataset["ability"].astype("category")

# Encode the categorical 'Rappel' column as numerical codes
dataset["ability"] = dataset["ability"].cat.codes

# Extract the email content and labels into separate variables
texts = dataset.initial_prompt.values
labels = dataset.ability.values

In [24]:
from tqdm import tqdm

# Define the path to the pre-trained model
model = "/home/snt/llm_models/bert-base-multilingual-uncased"

# Initialize the tokenizer from the pre-trained model, with lowercasing enabled
tokenizer = BertTokenizer.from_pretrained(model, do_lower_case=True)

# Get the maximum token length supported by the tokenizer
max_token_length = tokenizer.model_max_length

# Initialize a variable to keep track of the maximum length of tokenized input
max_len = 0

# Iterate over each text in the dataset
for text in tqdm(texts, desc="Processing texts"):
    # Tokenize the text and add special tokens
    input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True)
    # Update max_len with the length of the current tokenized input if it's greater than the current max_len
    max_len = max(max_len, len(input_ids))

# Determine the maximum padding length
if max_len < max_token_length:
    max_padding = max_len
else:
    max_padding = max_token_length

max_padding  # Need to padding to 512

Processing texts: 100%|██████████| 30214/30214 [02:29<00:00, 202.76it/s]


512

In [25]:
input_ids = []
attention_masks = []

for text in tqdm(texts, desc="Encoding texts"):
    # `encode_plus` (1) Tokenize the sentence. (2) Prepend the `[CLS]` token to the start. (3) Append the `[SEP]` token to the end.
    #  (4) Map tokens to their IDs. (5) Pad or truncate the sentence to `max_length` (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
        text,  # Sentence to encode.
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=max_padding,  # Pad & truncate all sentences.
        pad_to_max_length=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors="pt",  # Return pytorch tensors.
    )

    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])

# Convert the lists into tensors. [Samples, max_length]
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels).to(torch.int64)
labels = torch.nn.functional.one_hot(
    labels
).float()  # Change to Float in order to calcualte Loss [Samples, 2]

Encoding texts:   0%|          | 0/30214 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/snt/miniconda3/envs/causal_env_transformer/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Enco

In [26]:
# Combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Set the random seed for reproducibility and shuffle the dataset
torch.manual_seed(40)  # Orginally set to 42 for training
sample_indices = torch.randperm(len(dataset))
test_dataset = torch.utils.data.Subset(dataset, sample_indices)

# Calculate the sizes for training and validation datasets
train_size = int(0.6 * len(test_dataset))
val_size = len(test_dataset) - train_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(test_dataset, [train_size, val_size])

# Print the number of samples in the training and validation sets
print("{:>5,} training samples".format(train_size))
print("{:>5,} validation samples".format(val_size))

# Dataset preparation
batch_size = 96
train_dataloader = DataLoader(
    train_dataset,  # The training samples.
    sampler=RandomSampler(train_dataset),  # Select batches randomly
    batch_size=batch_size,  # Trains with this batch size.
)

validation_dataloader = DataLoader(
    val_dataset,  # The validation samples.
    sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
    batch_size=batch_size,  # Evaluate with this batch size.
)

18,128 training samples
12,086 validation samples


## Training Engine Initialization

In [ ]:
model = "/home/snt/llm_models/bert-base-multilingual-uncased"

model = BertForSequenceClassification.from_pretrained(
    model,  # Use the 12-layer BERT model, with an uncased vocab.
    num_labels=6,  # The number of output labels--2 for binary classification.
    output_attentions=False,  # Whether the model returns attentions weights.
    output_hidden_states=False,  # Whether the model returns all hidden-states.
)

model = model.to(device)


optimizer = AdamW(
    model.parameters(),
    lr=1e-6,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
    eps=1e-8,  # args.adam_epsilon  - default is 1e-8.
)

# Number of training epochs.
epochs = 10

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler. (This is important to decrease learning while training)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps,
)


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = np.argmax(labels, axis=1).flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

## Start Training

In [28]:
from sklearn.utils.class_weight import compute_class_weight

b_labels_np = labels.cpu().numpy()

labels_np = np.argmax(b_labels_np, axis=1)  

unique_labels = np.unique(labels_np) 
class_weights = compute_class_weight(
    class_weight="balanced", classes=unique_labels, y=labels_np
)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

In [29]:
# from torch.utils.data import DataLoader, Subset

# dataset = train_dataloader.dataset  # Getting the underlying dataset of the DataLoader

# # Shuffle the dataset and select 100 random indices
# random_indices = random.sample(range(len(dataset)), 100)

# # Create a Subset of the dataset with the selected indices
# test_subset = Subset(dataset, random_indices)

# # Create a DataLoader for the 100 samples
# test_dataloader = DataLoader(test_subset, batch_size=32, shuffle=False)  # Adjust batch_size if necessary

# dataset = validation_dataloader.dataset  # Getting the underlying dataset of the DataLoader

# # Shuffle the dataset and select 100 random indices
# random_indices = random.sample(range(len(dataset)), 100)

# # Create a Subset of the dataset with the selected indices
# test_subset = Subset(dataset, random_indices)

# Create a DataLoader for the 100 samples
# val_test_dataloader = DataLoader(test_subset, batch_size=32, shuffle=False)  # Adjust batch_size if necessary

In [30]:
seed_val = 43  # Set the seed value for reproducibility originally 42
random.seed(seed_val)  # Set the seed for the Python random module
np.random.seed(seed_val)  # Set the seed for NumPy's random number generator
torch.manual_seed(seed_val)  # Set the seed for PyTorch's random number generator
torch.cuda.manual_seed_all(
    seed_val
)  # Set the seed for all CUDA (GPU) operations in PyTorch


training_stats = []  # Initialize an empty list to store training statistics
total_t0 = time.time()  # Measure the total training time for the whole run.

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print("======== Epoch {:} / {:} ========".format(epoch_i + 1, epochs))
    print("Training...")
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(
        tqdm(train_dataloader, desc="Training Epoch {}/{}".format(epoch_i + 1, epochs))
    ):
        # Unpack this training batch from our dataloader.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels,
        )
        loss = output.loss  # Loss is calculate by forwarding

        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
        loss = output.loss
        labels_int = torch.argmax(b_labels, dim=1)
        loss = loss * class_weights_tensor[labels_int].mean()

        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(
            model,
            f"/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/training/bert_model_target_",
        )
        best_eval_accuracy = avg_val_accuracy
    torch.save(
        model,
        f"/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/training/bert_model_target_{epoch_i}",
    )
    # print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    # print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            "epoch": epoch_i + 1,
            "Training Loss": avg_train_loss,
            "Valid. Loss": avg_val_loss,
            "Valid. Accur.": avg_val_accuracy,
            "Training Time": training_time,
            "Validation Time": validation_time,
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 10 ========
Training...


Training Epoch 1/10:   0%|          | 0/189 [00:00<?, ?it/s]

Training Epoch 1/10: 100%|██████████| 189/189 [05:42<00:00,  1.81s/it]



  Average training loss: 0.56
  Training epcoh took: 0:05:43

Running Validation...
  Accuracy: 0.73

======== Epoch 2 / 10 ========
Training...


Training Epoch 2/10: 100%|██████████| 189/189 [05:47<00:00,  1.84s/it]


...


  Average training loss: 0.17
  Training epcoh took: 0:05:47

Running Validation...
  Accuracy: 0.95

======== Epoch 9 / 10 ========
Training...


Training Epoch 9/10: 100%|██████████| 189/189 [05:47<00:00,  1.84s/it]



  Average training loss: 0.17
  Training epcoh took: 0:05:48

Running Validation...
  Accuracy: 0.95

======== Epoch 10 / 10 ========
Training...


Training Epoch 10/10: 100%|██████████| 189/189 [05:47<00:00,  1.84s/it]



  Average training loss: 0.17
  Training epcoh took: 0:05:47

Running Validation...
  Accuracy: 0.95

Training complete!
Total training took 1:11:35 (h:mm:ss)
